In [1]:
from model.RNINet import *
from model.network_swinir_mask import SwinIR
from train_lab.train_base import *
from data.dataset_multi_noise import *

In [2]:
def define_model(model_name):
    if model_name=="RNINet":
        net=RNINet(apply_prob=0.5,ep=1e-6,in_nc=3, out_nc=3,act_mode='BR')
        pretrained_model = torch.load("model_zoo/checkpoint.pth")
        net.load_state_dict(pretrained_model, strict=True)
    elif model_name=="MaskTraining":
        opt_net= {
              "net_type": "swinir"
              , "upscale": 1
              , "in_chans": 3
              , "img_size": 64
              , "window_size": 8
              , "img_range": 1.0
              , "depths": [6, 6, 6, 6]
              , "embed_dim": 60
              , "num_heads": [6, 6, 6, 6]
              , "mlp_ratio": 2
              , "upsampler": None        
              , "resi_connection": "3conv"        

              , "init_type": "default"

              , "talking_heads": False
              , "attn_fn": "softmax"
              , "head_scale": False
              , "on_attn": False

              , "use_mask": True         
              , "mask_ratio1": 75        
              , "mask_ratio2": 75        
              , "mask_is_diff": False

              , "type": "stand"

            }
        net = SwinIR(upscale=opt_net['upscale'],
                           in_chans=opt_net['in_chans'],
                           img_size=opt_net['img_size'],
                           window_size=opt_net['window_size'],
                           img_range=opt_net['img_range'],
                           depths=opt_net['depths'],
                           embed_dim=opt_net['embed_dim'],
                           num_heads=opt_net['num_heads'],
                           mlp_ratio=opt_net['mlp_ratio'],
                           upsampler=opt_net['upsampler'],
                           resi_connection=opt_net['resi_connection'],
                           talking_heads=opt_net['talking_heads'], 
                           use_attn_fn=opt_net['attn_fn'],                   
                           head_scale=opt_net['head_scale'],                   
                           on_attn=opt_net['on_attn'],     
                           use_mask=opt_net['use_mask'],     
                           mask_ratio1=opt_net['mask_ratio1'],     
                           mask_ratio2=opt_net['mask_ratio2'],     
                           mask_is_diff=opt_net['mask_is_diff'],     
                           type=opt_net['type'],     
                           opt=opt_net,     
                           )
        pretrained_model = torch.load("model_zoo/input_mask_80_90.pth")
        net.load_state_dict(pretrained_model, strict=True)
    return net

In [8]:
net=define_model("RNINet")
device="cuda:0"
net.eval()
net.to(device)

In [4]:
data_path="home/testsets/McMaster/"
n_channels=3
alpha=1.5
label="test"
#label="MC_mask_test"
logger_info(label,label+".log")

LogHandlers setup!


In [5]:
noise_opt={"noise_type":"Poisson Noise","alpha":alpha}
test_loader=create_dataset(data_path,n_channels,noise_opt)

Dataset: MultiNoise. Only dataroot_H is needed.


In [7]:
test_backbone(net,device, label+"_"+str(alpha),"123456",test_loader,label)
#model,device_name, testset,iter_time,test_loader,logger_name

25-07-13 14:59:20.137 : testset test_1.5, iter_time 123456, avg psnr/ssim is 33.65/0.9067
